## Assessing allelic bias in RADseq data

Can we identify 'bad' loci by looking at allelic bias in heterozygotes?

Within a *heterozygous* individual we can assess allelic bias at a specific locus with a binomial test.  The null hypothesis is that each allelic sequence is equally likely. The p value represents the likelihood of a bias at least as big as observed under the null. Notice this is a two-tailed test.

In [1]:
import scipy.stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels
import numpy as np
import os.path
import gzip

In [2]:
%matplotlib inline

## Example bionimial test

In [3]:
coverage_allele_a = 10
coverage_allele_b = 30

p_value = scipy.stats.binom_test((coverage_allele_a,coverage_allele_b))
print(p_value)

0.00222143377323


Caveat: due to PCR duplicates the reads may not be **truly** independent, hopefully this effect is not allele-specific.

### Assessing one locus across multiple individuals

By observing allele bias at a target locus across multiple individuals we can try to see if there is a **consistent** bias.  

This raises the question of how to combine data from many individuals, I see two distinct options:
   1. weight each **allele observation** equally, essentially ignoring the indidviual that they appear in.  This implicitly assumes that any process generating bias operates somewhat cosistently across individuals.
   2. weight each **individual** equally, this accounts for the difference in coverage across individuals, ignoring the fact that individuals with difference coverages provide more information on allelic bias.
   
   OR 
   3. some combination the two.

#### In case 1. we can simply add up all the observations of the 'a' alelle and all observations of the 'b' allele and run the same binomial test.

In [4]:
coverage_allele_a_accross_individuals = (25,13,2,3,7,17,9,29,51,2,4,29,14)
coverage_allele_a = sum(coverage_allele_a_accross_individuals)
print('coverage_allele_a: {}'.format(coverage_allele_a))

coverage_allele_b_accross_individuals = (28,23,22,1,17,14,19,5,27,12,8,21,19)
coverage_allele_b = sum(coverage_allele_b_accross_individuals)
print('coverage_allele_a: {}'.format(coverage_allele_b))

p_value = scipy.stats.binom_test((coverage_allele_a,coverage_allele_b))
print(p_value)

coverage_allele_a: 205
coverage_allele_a: 216
0.626048377669


#### In case 2. we keep each individual separate.

In [5]:
ab = zip(coverage_allele_a_accross_individuals, coverage_allele_b_accross_individuals)
p_vals = [scipy.stats.binom_test(xx) for xx in ab]
p_vals

[0.78384630220949836,
 0.13249816396273675,
 3.5881996154785156e-05,
 0.625,
 0.063914656639099121,
 0.72010013181716115,
 0.087158553302288042,
 3.8558151572942747e-05,
 0.0087717144570601362,
 0.012939453124999998,
 0.38769531250000011,
 0.32223632035754712,
 0.4868502416647969]

combine the p values into one measure

Fisher's method

In [6]:
fisher_statistic, fisher_pval = scipy.stats.combine_pvalues(p_vals, method = 'fisher')
fisher_statistic, fisher_pval

(81.070899768715066, 1.4321730977438808e-07)

Stouffer's method.



In [7]:
stouffer_statistic, stouffer_pval = scipy.stats.combine_pvalues(p_vals, method = 'stouffer', weights = None) # can be weighted
stouffer_statistic, stouffer_pval

(4.3308924674047935, 7.4253098511083297e-06)

In this method you can weight the individuals.

In [8]:
my_weights = [np.sqrt(sum([a,b])) for a,b in ab]
stouffer_statistic, stouffer_pval = scipy.stats.combine_pvalues(p_vals, method = 'stouffer', weights = my_weights) # can be weighted
stouffer_statistic, stouffer_pval

(4.2883981531672024, 8.9983142308648029e-06)